In [32]:
import numpy as np
import pandas as pd
import matplotlib as plt

Train_Inpatient = pd.read_csv('Train_Inpatientdata-1542865627584.csv')
Test_Inpatient = pd.read_csv('Test_Inpatientdata-1542969243754.csv')
Test_Inpatient.index = Test_Inpatient.index + len(Train_Inpatient.index)
Inpatient = pd.concat([Train_Inpatient, Test_Inpatient], axis=0)

Train_Outpatient = pd.read_csv('Train_Outpatientdata-1542865627584.csv')
Test_Outpatient = pd.read_csv('Test_Outpatientdata-1542969243754.csv')
Test_Outpatient.index = Test_Outpatient.index + len(Train_Outpatient.index)
Outpatient = pd.concat([Train_Outpatient, Test_Outpatient], axis=0)

Train_Beneficiary = pd.read_csv('Train_Beneficiarydata-1542865627584.csv')
Test_Beneficiary = pd.read_csv('Test_Beneficiarydata-1542969243754.csv')
Test_Beneficiary.index = Test_Beneficiary.index + len(Train_Beneficiary.index)
Beneficiary = pd.concat([Train_Beneficiary, Test_Beneficiary], axis=0)
Beneficiary = Beneficiary.drop_duplicates()

Train_Fraud_Samples = pd.read_csv('Train-1542865627584.csv')
Test_Fraud_Samples = pd.read_csv('Test-1542969243754.csv')
Test_Fraud_Samples.index = Test_Fraud_Samples.index + len(Train_Fraud_Samples.index)
Fraud_Samples = pd.concat([Train_Fraud_Samples, Test_Fraud_Samples], axis=0)

In [28]:
print(Train_Inpatient.shape, Train_Inpatient.columns)
print(Train_Outpatient.shape, Train_Outpatient.columns)
print(Train_Beneficiary.shape, Train_Beneficiary.columns)

(40474, 30) Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6'],
      dtype='object')
(517737, 27) Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosis

In [33]:
print(Inpatient.shape, Train_Inpatient.shape[0] + Test_Inpatient.shape[0])
print(Outpatient.shape, Train_Outpatient.shape[0] + Test_Outpatient.shape[0])
print(Beneficiary.shape, Train_Beneficiary.shape[0] + Test_Beneficiary.shape[0])

(50025, 30) 50025
(643578, 27) 643578
(202524, 25) 202524


In [34]:
Inpatient['InpatientAdmission'] = 1
Outpatient['InpatientAdmission'] = 0

Only admitted if inpatient, so 'Length of Stay' calculated with Inpatient dataset

In [35]:
Inpatient['Length_of_Stay'] = pd.to_datetime(Inpatient['DischargeDt']) - pd.to_datetime(Inpatient['AdmissionDt'])
Inpatient['Length_of_Stay'] = Inpatient['Length_of_Stay'].astype(str).str.extract('(\d+)').astype(int) + 1

Inpatient['Reimbursement_per_Day'] = Inpatient['InscClaimAmtReimbursed'] / Inpatient['Length_of_Stay']

Inpatient['MismatchedAdmitCode'] = np.where(Inpatient['ClmAdmitDiagnosisCode'] == Inpatient['ClmDiagnosisCode_1'], 0, 1)

In [51]:
All_Visits = Inpatient.append(Outpatient).set_index('ClaimID').reset_index()
print(All_Visits.shape, Inpatient.shape[0] + Outpatient.shape[0])

(693603, 34) 693603


In [52]:
All_Visits

,ClaimID,BeneID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,InpatientAdmission,Length_of_Stay,Reimbursement_per_Day,MismatchedAdmitCode
0,CLM46614,BENE11001,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,NaN,NaN,NaN,NaN,NaN,NaN,1,7.0,3714.285714,1.0
1,CLM66048,BENE11001,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,7092.0,NaN,NaN,NaN,NaN,NaN,1,3.0,1666.666667,0.0
2,CLM68358,BENE11001,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,1,4.0,1250.000000,1.0
3,CLM38412,BENE11011,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,331.0,NaN,NaN,NaN,NaN,NaN,1,9.0,555.555556,1.0
4,CLM63689,BENE11014,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,3893.0,NaN,NaN,NaN,NaN,NaN,1,18.0,555.555556,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693598,CLM469576,BENE159193,2009-07-14,2009-07-14,PRV55485,70,PHY431265,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
693599,CLM483842,BENE159193,2009-07-22,2009-07-22,PRV55485,10,PHY369098,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
693600,CLM554925,BENE159193,2009-08-31,2009-08-31,PRV55485,30,PHY431298,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
693601,CLM347777,BENE159194,2009-05-08,2009-05-08,PRV55889,30,PHY406767,NaN,PHY406767,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


In [36]:
# All = pd.merge(Inpatient.append(Outpatient), Fraud_Samples, on='Provider', how='left')

In [38]:
# print(All.shape, Inpatient.shape[0] + Outpatient.shape[0])

(693603, 35) 693603


In [39]:
# print(All.shape, All.columns)

(693603, 35) Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6', 'InpatientAdmission', 'Length_of_Stay',
       'Reimbursement_per_Day', 'MismatchedAdmitCode', 'PotentialFraud'],
      dtype='object')


## Create Master Dataset with Inpatient and Outpatient Visits

In [53]:
All_Visits['Length_of_Claim'] = pd.to_datetime(All_Visits['ClaimEndDt']) - pd.to_datetime(All_Visits['ClaimStartDt'])
All_Visits['Length_of_Claim'] = All_Visits['Length_of_Claim'].astype(str).str.extract('(\d+)').astype(int) + 1

All_Visits['MonthofClaimStartDt'] = pd.to_datetime(All_Visits['ClaimStartDt']).dt.month_name()
All_Visits['WhichThirdofMonth'] = np.where(pd.to_datetime(All_Visits['ClaimStartDt']).dt.day > 10, np.where(pd.to_datetime(All_Visits['ClaimStartDt']).dt.day > 20, 3, 2), 1)

Docs_List = [col for col in All_Visits.columns if 'Physician' in col]
All_Visits['Num_Physicians_in_Claim'] = All_Visits[Docs_List].count(axis=1)

Claim_Procedure_List = [col for col in All_Visits.columns if 'ClmProcedureCode_' in col]
All_Visits['Num_ClmProcedureCodes'] = All_Visits[Claim_Procedure_List].count(axis=1)

Claim_Diagnosis_List = [col for col in All_Visits.columns if 'ClmDiagnosisCode_' in col]
All_Visits['Num_ClmDiagnosisCodes'] = All_Visits[Claim_Diagnosis_List].count(axis=1)

All_Visits['Num_ClmCodes'] = All_Visits['Num_ClmProcedureCodes'] + All_Visits['Num_ClmDiagnosisCodes']

In [54]:
print(All_Visits.shape, All_Visits.columns)

(693603, 41) Index(['ClaimID', 'BeneID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6', 'InpatientAdmission', 'Length_of_Stay',
       'Reimbursement_per_Day', 'MismatchedAdmitCode', 'Length_of_Claim',
       'MonthofClaimStartDt', 'WhichThirdofMonth', 'Num_Physicians_in_Claim',
       'Num_ClmProcedureCodes', 'Num_ClmDiagnosisCodes', 'Num_ClmCodes'],
      dtype='object')


Deductible for inpatient relatively useless as well, as patients are hitting their max deductible with every inpatient admission (if collected). Likely not the case with outpatient, however.

In [56]:
print(Beneficiary.shape, Beneficiary.columns)

(202524, 25) Index(['BeneID', 'DOB', 'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator',
       'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt'],
      dtype='object')


In [62]:
Beneficiary.sort_values('BeneID')

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
83155,BENE100000,1938-03-01,NaN,1,1,0,49,430,12,12,...,2,2,1,2,2,2,0,0,120,30
176995,BENE100001,1939-08-01,NaN,1,1,0,33,420,12,12,...,2,2,1,1,2,1,0,0,2530,540
83156,BENE100001,1939-08-01,NaN,1,1,0,33,420,12,12,...,2,2,1,1,2,1,0,0,2530,540
83157,BENE100002,1938-09-01,NaN,2,2,0,33,20,12,12,...,1,1,1,2,2,1,12250,1068,1760,660
176996,BENE100002,1938-09-01,NaN,2,2,0,33,20,12,12,...,1,1,1,2,2,1,12250,1068,1760,660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176993,BENE99997,1929-07-01,NaN,2,1,0,22,60,12,12,...,2,1,1,1,1,2,0,0,1570,490
83152,BENE99997,1929-07-01,NaN,2,1,0,22,60,12,12,...,2,1,1,1,1,2,0,0,1570,490
83153,BENE99998,1929-06-01,NaN,1,1,0,45,910,12,12,...,2,1,1,2,2,2,0,0,1600,320
176994,BENE99999,1936-05-01,NaN,2,1,0,29,150,12,12,...,2,1,1,1,2,2,0,0,1270,690


In [73]:
All_Visits_Bene = pd.merge(All_Visits, Beneficiary, on='BeneID', how='left')
All_Visits_Bene

,ClaimID,BeneID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,CLM46614,BENE11001,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,1,1,1,2,1,1,36000,3204,60,70
1,CLM66048,BENE11001,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,1,1,1,2,1,1,36000,3204,60,70
2,CLM68358,BENE11001,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,1,1,1,2,1,1,36000,3204,60,70
3,CLM38412,BENE11011,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,1,1,2,2,1,1,5000,1068,250,320
4,CLM63689,BENE11014,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,1,2,1,2,2,2,21260,2136,120,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693598,CLM469576,BENE159193,2009-07-14,2009-07-14,PRV55485,70,PHY431265,NaN,NaN,NaN,...,2,2,2,2,2,2,0,0,110,50
693599,CLM483842,BENE159193,2009-07-22,2009-07-22,PRV55485,10,PHY369098,NaN,NaN,NaN,...,2,2,2,2,2,2,0,0,110,50
693600,CLM554925,BENE159193,2009-08-31,2009-08-31,PRV55485,30,PHY431298,NaN,NaN,NaN,...,2,2,2,2,2,2,0,0,110,50
693601,CLM347777,BENE159194,2009-05-08,2009-05-08,PRV55889,30,PHY406767,NaN,PHY406767,NaN,...,2,2,2,2,2,2,0,0,430,460


In [74]:
All_Visits_Bene['Bene_Age_at_ClaimStartDt'] = pd.to_datetime(All_Visits_Bene['ClaimStartDt']) - pd.to_datetime(All_Visits_Bene['DOB'])#.dt.year might be simpler
All_Visits_Bene['Bene_Age_at_ClaimStartDt'] = np.floor(All_Visits_Bene['Bene_Age_at_ClaimStartDt'].astype(str).str.extract('(\d+)').astype(int) / 365.24)
All_Visits_Bene['Bene_Age_at_ClaimStartDt']

0         66.0
1         66.0
2         66.0
3         94.0
4         71.0
          ... 
693598    72.0
693599    72.0
693600    72.0
693601    69.0
693602    79.0
Name: Bene_Age_at_ClaimStartDt, Length: 693603, dtype: float64

In [68]:
# np.sum(pd.to_datetime(Train_All_Bene['DOD']) <= pd.to_datetime(Train_All_Bene['ClaimEndDt']))

In [75]:
All_Visits_Bene['TotalAnnualReimbursementAmt'] = All_Visits_Bene['IPAnnualReimbursementAmt'] + All_Visits_Bene['OPAnnualReimbursementAmt']
All_Visits_Bene['TotalAnnualDeductibleAmt'] = All_Visits_Bene['IPAnnualDeductibleAmt'] + All_Visits_Bene['OPAnnualDeductibleAmt']
All_Visits_Bene['Perc_of_AnnualReimbursement'] = np.where(All_Visits_Bene['InpatientAdmission'] == 1, 
    100 * All_Visits_Bene['InscClaimAmtReimbursed'] / All_Visits_Bene['IPAnnualReimbursementAmt'],
    100 * All_Visits_Bene['InscClaimAmtReimbursed'] / All_Visits_Bene['OPAnnualReimbursementAmt'])
All_Visits_Bene['Perc_of_AnnualDeductible'] = np.where(All_Visits_Bene['InpatientAdmission'] == 1, 
    100 * All_Visits_Bene['DeductibleAmtPaid'] / All_Visits_Bene['IPAnnualDeductibleAmt'],
    100 * All_Visits_Bene['DeductibleAmtPaid'] / All_Visits_Bene['OPAnnualDeductibleAmt'])
All_Visits_Bene['Perc_Paid_by_Insurance'] = 100 * All_Visits_Bene['InscClaimAmtReimbursed'] / (All_Visits_Bene['InscClaimAmtReimbursed'] + All_Visits_Bene['DeductibleAmtPaid'])

In [78]:
# All_Visits_Bene[(All_Visits_Bene['TotalAnnualReimbursementAmt'] != All_Visits_Bene['OPAnnualReimbursementAmt']) & (All_Visits_Bene['TotalAnnualReimbursementAmt'] != All_Visits_Bene['IPAnnualReimbursementAmt'])]

In [79]:
print(All_Visits_Bene.shape, All_Visits_Bene.columns)

(693603, 71) Index(['ClaimID', 'BeneID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6', 'InpatientAdmission', 'Length_of_Stay',
       'Reimbursement_per_Day', 'MismatchedAdmitCode', 'Length_of_Claim',
       'MonthofClaimStartDt', 'WhichThirdofMonth', 'Num_Physicians_in_Claim',
       'Num_ClmProcedureCodes', 'Num_ClmDiagnosisCodes', 'Num_ClmCodes', 'DOB',
       'DOD', 'Gender', 'Race', 'RenalDisea

In [80]:
np.unique(All_Visits_Bene.RenalDiseaseIndicator) 

array(['0', 'Y'], dtype=object)

In [81]:
All_Visits_Bene.RenalDiseaseIndicator.replace('Y','1', inplace=True)
All_Visits_Bene.RenalDiseaseIndicator = pd.to_numeric(All_Visits_Bene.RenalDiseaseIndicator)
np.unique(All_Visits_Bene.RenalDiseaseIndicator) 

array([0, 1], dtype=int64)

In [84]:
Binary_list = list(All_Visits_Bene.columns[All_Visits_Bene.columns.str.contains("Chronic")]) + ['Gender']
Binary_list
for binary in Binary_list:
    All_Visits_Bene[binary].replace(2, 0, inplace=True)
Binary_list_updated = Binary_list + ['RenalDiseaseIndicator']
np.sum((All_Visits_Bene[Binary_list_updated] != 0) & (All_Visits_Bene[Binary_list_updated] != 1))

ChronicCond_Alzheimer              0
ChronicCond_Heartfailure           0
ChronicCond_KidneyDisease          0
ChronicCond_Cancer                 0
ChronicCond_ObstrPulmonary         0
ChronicCond_Depression             0
ChronicCond_Diabetes               0
ChronicCond_IschemicHeart          0
ChronicCond_Osteoporasis           0
ChronicCond_rheumatoidarthritis    0
ChronicCond_stroke                 0
Gender                             0
RenalDiseaseIndicator              0
dtype: int64

In [85]:
All_Visits_Bene['Num_ChronicCond'] = All_Visits_Bene['ChronicCond_Alzheimer'] + All_Visits_Bene['ChronicCond_Heartfailure'] + All_Visits_Bene['ChronicCond_KidneyDisease'] + All_Visits_Bene['ChronicCond_Cancer'] + All_Visits_Bene['ChronicCond_ObstrPulmonary'] + All_Visits_Bene['ChronicCond_Depression'] + All_Visits_Bene['ChronicCond_Diabetes'] + All_Visits_Bene['ChronicCond_IschemicHeart'] + All_Visits_Bene['ChronicCond_Osteoporasis'] + All_Visits_Bene['ChronicCond_rheumatoidarthritis'] + All_Visits_Bene['ChronicCond_stroke']
All_Visits_Bene['Num_ChronicCond']

0         7
1         7
2         7
3         6
4         5
         ..
693598    0
693599    0
693600    0
693601    1
693602    3
Name: Num_ChronicCond, Length: 693603, dtype: int64

In [87]:
# "0 = Unknown
# 1 = White
# 2 = Black
# 3 = Other
# 4 = Asian
# 5 = Hispanic
# 6 = North American Native
# "
All_Visits_Bene.Race.value_counts()
All_Visits_Bene['Race_White'] = np.where(All_Visits_Bene.Race == 1, 1, 0)
All_Visits_Bene['Race_NonWhite'] = np.where(All_Visits_Bene.Race != 1, 1, 0)
All_Visits_Bene['Race_Black'] = np.where(All_Visits_Bene.Race == 2, 1, 0)
All_Visits_Bene['Race_Other'] = np.where(All_Visits_Bene.Race == 3, 1, 0)
All_Visits_Bene['Race_Hispanic'] = np.where(All_Visits_Bene.Race == 5, 1, 0)

All_Visits_Bene['Age_Under_65'] = np.where(All_Visits_Bene.Bene_Age_at_ClaimStartDt < 65, 1, 0)
All_Visits_Bene['Age_65_upto_75'] = np.where((All_Visits_Bene.Bene_Age_at_ClaimStartDt >= 65) & (All_Visits_Bene.Bene_Age_at_ClaimStartDt < 75), 1, 0)
All_Visits_Bene['Age_75_upto_85'] = np.where((All_Visits_Bene.Bene_Age_at_ClaimStartDt >= 75) & (All_Visits_Bene.Bene_Age_at_ClaimStartDt < 85), 1, 0)
All_Visits_Bene['Age_85_or_Over'] = np.where(All_Visits_Bene.Bene_Age_at_ClaimStartDt >= 85, 1, 0)

In [90]:
Bene_Char = ['BeneID','Provider', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician']
BeneData = All_Visits_Bene[Bene_Char].groupby('BeneID')

In [95]:
def list_(df):
    return list(df[df.notnull()])

def max_(df):
    try: 
        max_val = df.max()
    except:
        max_val = np.nan
    return max_val

def len_value_count(df):
    return len(df.value_counts())

def avg(df):
    return round(df.mean(skipna=True),2)

# def value_count(df):
#     return [list(df.value_counts().index), list(df.value_counts())]
# def value_count_highest(df):
#     return list(df.value_counts())[0]
# def value_count_list(df):
#     return list(df.value_counts().index)
# def num_unique(df):
#     return len(np.unique(df[~np.isnan(df)]))

In [96]:
Docs = BeneData[Docs_List].agg(value_count_list)

Docs['PhysicianList'] = Docs['AttendingPhysician'] + Docs['OperatingPhysician'] + Docs['OtherPhysician']
Docs['Num_Physicians_Seen'] = Docs['PhysicianList'].apply(lambda x: len(np.unique(x)))
Docs['Mean_Num_Physicians_in_Claim'] = All_Visits_Bene[['Num_Physicians_in_Claim', 'BeneID']].groupby('BeneID').agg('mean')

Docs['PhysList'] = BeneData['AttendingPhysician'].agg(list_) + BeneData['OperatingPhysician'].agg(list_) + BeneData['OtherPhysician'].agg(list_)

Docs['Most_Num_Visits_Same_Phys'] = [max_(pd.Series(x).value_counts()) for x in Docs.PhysList.values]
Docs['Avg_Num_Visits_Same_Phys'] = [avg(pd.Series(x).value_counts()) for x in Docs.PhysList.values]

Docs['Num_Providers_Seen'] = BeneData['Provider'].agg(len_value_count)
Docs['Num_Visits'] = BeneData['Provider'].agg('count')

BeneGroup = All_Visits_Bene[['InpatientAdmission', 'BeneID']].groupby('BeneID')
Docs['Num_Inpatient_Visits'] = BeneGroup['InpatientAdmission'].agg('sum')
Docs['Num_Outpatient_Visits'] = BeneGroup['InpatientAdmission'].agg('count') - BeneGroup['InpatientAdmission'].agg('sum')

Docs

C:\Users\gottl\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
C:\Users\gottl\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.


,AttendingPhysician,OperatingPhysician,OtherPhysician,PhysicianList,Num_Physicians_Seen,Mean_Num_Physicians_in_Claim,PhysList,Most_Num_Visits_Same_Phys,Avg_Num_Visits_Same_Phys,Num_Providers_Seen,Num_Visits,Num_Inpatient_Visits,Num_Outpatient_Visits
BeneID,,,,,,,,,,,,,
BENE100000,"[PHY370909, PHY383401]",[],[PHY363377],"[PHY370909, PHY383401, PHY363377]",3,1.500000,"[PHY383401, PHY370909, PHY363377]",1.0,1.00,1,2,0,2
BENE100001,"[PHY425443, PHY343317, PHY364771, PHY383478, P...","[PHY313692, PHY408367, PHY390959]","[PHY337051, PHY408367, PHY349909, PHY418077]","[PHY425443, PHY343317, PHY364771, PHY383478, P...",16,1.636364,"[PHY383478, PHY347511, PHY343317, PHY425169, P...",2.0,1.12,7,11,0,11
BENE100002,"[PHY373032, PHY332965, PHY317949, PHY365254, P...","[PHY390246, PHY373032, PHY352364, PHY323686, P...","[PHY386539, PHY358567, PHY315572]","[PHY373032, PHY332965, PHY317949, PHY365254, P...",17,1.533333,"[PHY424317, PHY327219, PHY373032, PHY317949, P...",5.0,1.35,12,15,1,14
BENE100003,[PHY347780],[],[],[PHY347780],1,1.000000,[PHY347780],1.0,1.00,1,1,0,1
BENE100004,"[PHY418388, PHY383988, PHY319940, PHY364921, P...",[],"[PHY418388, PHY319940, PHY364921]","[PHY418388, PHY383988, PHY319940, PHY364921, P...",6,1.500000,"[PHY319940, PHY410241, PHY383988, PHY418388, P...",4.0,2.00,6,8,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
BENE99995,"[PHY412126, PHY363575]",[PHY338931],[PHY412126],"[PHY412126, PHY363575, PHY338931, PHY412126]",3,2.000000,"[PHY412126, PHY363575, PHY338931, PHY412126]",2.0,1.33,2,2,1,1
BENE99996,[PHY339956],[],[],[PHY339956],1,1.000000,[PHY339956],1.0,1.00,1,1,0,1
BENE99997,"[PHY351183, PHY389189, PHY314880, PHY340798, P...",[],[PHY359055],"[PHY351183, PHY389189, PHY314880, PHY340798, P...",7,1.125000,"[PHY340798, PHY338082, PHY389189, PHY351183, P...",2.0,1.29,5,8,0,8


In [97]:
import pickle

with open('Docs_stats.pickle','wb') as f:
     pickle.dump(Docs, f)

In [98]:
Important = ['Mean_Num_Physicians_in_Claim', 'Most_Num_Visits_Same_Phys', 'Avg_Num_Visits_Same_Phys', 'Num_Physicians_Seen', 'Num_Providers_Seen', 'Num_Visits', 
             'Num_Inpatient_Visits', 'Num_Outpatient_Visits']
Important_Bene = Docs[Important]
All_Visits_Bene_Important = pd.merge(All_Visits_Bene, Important_Bene, on='BeneID', how='left')
All_Data = All_Visits_Bene_Important.groupby('Provider')

In [111]:
Xtrain = pd.DataFrame(All_Data['ClaimID'].agg('count'))
Xtrain.columns = ['Number_of_Claims']

In [112]:
def unique_(df):
    return len(df.value_counts())

Xtrain['Num_Unique_Patients'] = All_Data['BeneID'].agg(unique_)
Xtrain

,Number_of_Claims,Num_Unique_Patients
Provider,,
PRV51001,25,24
PRV51002,205,169
PRV51003,132,117
PRV51004,149,138
PRV51005,1165,495
...,...,...
PRV57759,28,24
PRV57760,22,9
PRV57761,82,67


In [113]:
DocsList = ['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']
DocData = All_Data[DocsList].agg(value_count_list)
DocData['DocDataList'] = DocData['AttendingPhysician'] + DocData['OperatingPhysician'] + DocData['OtherPhysician']
Xtrain['Num_Unique_Docs'] = DocData['DocDataList'].apply(lambda x: len(np.unique(x)))

ClmDiagList = list(All_Visits_Bene_Important.columns[All_Visits_Bene_Important.columns.str.contains("ClmDiagnosisCode_")])
Diag = All_Data[ClmDiagList].agg(value_count_list)
Diag['DiagList'] = Diag['ClmDiagnosisCode_1'] + Diag['ClmDiagnosisCode_2'] + Diag['ClmDiagnosisCode_3'] + Diag['ClmDiagnosisCode_4'] + Diag['ClmDiagnosisCode_5'] + Diag['ClmDiagnosisCode_6'] + Diag['ClmDiagnosisCode_7'] + Diag['ClmDiagnosisCode_8'] + Diag['ClmDiagnosisCode_9'] + Diag['ClmDiagnosisCode_10']
Xtrain['Num_Unique_Diag_Codes'] = Diag['DiagList'].apply(lambda x: len(np.unique(x)))

ClmProcList = list(All_Visits_Bene_Important.columns[All_Visits_Bene_Important.columns.str.contains("ClmProcedureCode_")])
Proc = All_Data[ClmProcList].agg(value_count_list)
Proc['ProcList'] = Proc['ClmProcedureCode_1'] + Proc['ClmProcedureCode_2'] + Proc['ClmProcedureCode_3'] + Proc['ClmProcedureCode_4'] + Proc['ClmProcedureCode_5'] + Proc['ClmProcedureCode_6']
Xtrain['Num_Unique_Proc_Codes'] = Proc['ProcList'].apply(lambda x: len(np.unique(x)))

In [114]:
Xtrain

,Number_of_Claims,Num_Unique_Patients,Num_Unique_Docs,Num_Unique_Diag_Codes,Num_Unique_Proc_Codes
Provider,,,,,
PRV51001,25,24,19,72,3
PRV51002,205,169,94,361,0
PRV51003,132,117,51,365,39
PRV51004,149,138,48,267,0
PRV51005,1165,495,7,1294,0
...,...,...,...,...,...
PRV57759,28,24,1,49,0
PRV57760,22,9,3,47,0
PRV57761,82,67,7,182,0


In [118]:
mean_list = ['InscClaimAmtReimbursed', 'DeductibleAmtPaid', 
       'Reimbursement_per_Day', 'Length_of_Stay', 'Length_of_Claim',
       'WhichThirdofMonth', 'Num_Physicians_in_Claim', 'Num_ClmProcedureCodes', 'Num_ClmDiagnosisCodes',
       'Num_ClmCodes', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
        'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt', 'Bene_Age_at_ClaimStartDt',
       'TotalAnnualReimbursementAmt', 'TotalAnnualDeductibleAmt',
       'Perc_of_AnnualReimbursement', 'Perc_of_AnnualDeductible',
       'Perc_Paid_by_Insurance', 'Num_ChronicCond', 
       'Num_Physicians_Seen', 'Most_Num_Visits_Same_Phys', 
       'Avg_Num_Visits_Same_Phys', 'Num_Providers_Seen', 'Num_Visits',
       'Num_Inpatient_Visits', 'Num_Outpatient_Visits']

perc_list = ['MismatchedAdmitCode', 'InpatientAdmission', 'RenalDiseaseIndicator', 'ChronicCond_Alzheimer', 
             'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
             'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression','ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
             'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke', 'Age_Under_65',
             'Age_65_upto_75', 'Age_75_upto_85', 'Age_85_or_Over', 'Race_White',
             'Race_NonWhite', 'Race_Black', 'Race_Other', 'Race_Hispanic']
mean_names = ['Mean_' + x for x in mean_list]
perc_names = ['Perc_' + x for x in perc_list]
Xtrain_mean = All_Data[mean_list].agg(avg)
Xtrain_mean.columns = mean_names
Xtrain_perc = All_Data[perc_list].agg(avg)
Xtrain_perc.columns = perc_names

In [119]:
Claim_Month = pd.get_dummies(list(All_Visits_Bene_Important['MonthofClaimStartDt']), prefix='ClaimStartDt', prefix_sep='_') 
Claim_Month = pd.concat([pd.DataFrame(All_Visits_Bene_Important['Provider']), Claim_Month], axis=1)
Claim_Month_Group = Claim_Month.groupby('Provider').agg(avg)
monthsinorder = ['ClaimStartDt_January', 'ClaimStartDt_February', 'ClaimStartDt_March', 'ClaimStartDt_April', 
                 'ClaimStartDt_May', 'ClaimStartDt_June', 'ClaimStartDt_July', 'ClaimStartDt_August',
                 'ClaimStartDt_September', 'ClaimStartDt_October', 'ClaimStartDt_November', 'ClaimStartDt_December']
Claim_Month_Group = Claim_Month_Group.filter(monthsinorder)
month_names = ['Perc_' + x for x in monthsinorder]
Claim_Month_Group.columns = month_names

In [120]:
Xtrain = pd.concat([Xtrain, Xtrain_mean], axis=1)
Xtrain['Perc_Gender_Male'] = All_Data['Gender'].agg(avg)
Xtrain = pd.concat([Xtrain, pd.concat([Xtrain_perc, Claim_Month_Group], axis=1)], axis=1) 
Xtrain

,Number_of_Claims,Num_Unique_Patients,Num_Unique_Docs,Num_Unique_Diag_Codes,Num_Unique_Proc_Codes,Mean_InscClaimAmtReimbursed,Mean_DeductibleAmtPaid,Mean_Reimbursement_per_Day,Mean_Length_of_Stay,Mean_Length_of_Claim,...,Perc_ClaimStartDt_March,Perc_ClaimStartDt_April,Perc_ClaimStartDt_May,Perc_ClaimStartDt_June,Perc_ClaimStartDt_July,Perc_ClaimStartDt_August,Perc_ClaimStartDt_September,Perc_ClaimStartDt_October,Perc_ClaimStartDt_November,Perc_ClaimStartDt_December
Provider,,,,,,,,,,,,,,,,,,,,,
PRV51001,25,24,19,72,3,4185.60,213.60,3700.00,6.00,2.44,...,0.12,0.08,0.20,0.12,0.08,0.04,0.04,0.00,0.00,0.08
PRV51002,205,169,94,361,0,262.39,1.85,NaN,NaN,1.98,...,0.09,0.10,0.08,0.06,0.08,0.08,0.06,0.10,0.10,0.08
PRV51003,132,117,51,365,39,4588.41,502.17,2137.42,6.16,4.67,...,0.08,0.08,0.06,0.11,0.08,0.10,0.07,0.08,0.08,0.05
PRV51004,149,138,48,267,0,350.13,2.08,NaN,NaN,2.43,...,0.12,0.07,0.05,0.07,0.09,0.07,0.08,0.07,0.05,0.11
PRV51005,1165,495,7,1294,0,241.12,3.18,NaN,NaN,2.09,...,0.10,0.08,0.07,0.09,0.08,0.08,0.09,0.10,0.06,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PRV57759,28,24,1,49,0,380.00,4.64,NaN,NaN,3.14,...,0.07,0.07,0.14,0.11,0.11,0.04,0.07,0.04,0.07,0.04
PRV57760,22,9,3,47,0,216.82,0.00,NaN,NaN,1.32,...,0.05,0.05,0.05,0.14,0.00,0.18,0.14,0.00,0.09,0.05
PRV57761,82,67,7,182,0,225.24,4.51,NaN,NaN,2.39,...,0.05,0.06,0.10,0.12,0.06,0.06,0.05,0.17,0.13,0.06


Eliminating Null Values

In [122]:
np.sum(Xtrain.isna()).sort_values(ascending=False).head(10)

Mean_Length_of_Stay                 4151
Perc_MismatchedAdmitCode            4151
Mean_Reimbursement_per_Day          4151
Mean_Perc_Paid_by_Insurance           15
Mean_Perc_of_AnnualDeductible         11
Mean_DeductibleAmtPaid                 1
Mean_Num_ChronicCond                   0
Mean_Bene_Age_at_ClaimStartDt          0
Mean_TotalAnnualReimbursementAmt       0
Mean_TotalAnnualDeductibleAmt          0
dtype: int64

In [125]:
# These patients did not have an inpatient stay #
Xtrain.Mean_Length_of_Stay.fillna(0, inplace=True)
Xtrain.Perc_MismatchedAdmitCode.fillna(0, inplace=True)
Xtrain.Mean_Reimbursement_per_Day.fillna(0, inplace=True)

Xtrain.Mean_Perc_Paid_by_Insurance.fillna(round(100 * (Xtrain.Mean_IPAnnualReimbursementAmt + Xtrain.Mean_OPAnnualReimbursementAmt) / Xtrain.Mean_TotalAnnualReimbursementAmt), inplace=True)
Xtrain.Mean_Perc_of_AnnualDeductible.fillna(round(100 * (Xtrain.Mean_IPAnnualDeductibleAmt + Xtrain.Mean_OPAnnualDeductibleAmt) / Xtrain.Mean_TotalAnnualDeductibleAmt), inplace=True)
Xtrain.Mean_Perc_of_AnnualDeductible.fillna(round(np.median(Xtrain.Mean_Perc_of_AnnualDeductible[~Xtrain.Mean_Perc_of_AnnualDeductible.isna()])), inplace=True)
Xtrain.Mean_DeductibleAmtPaid.fillna(round(np.median(Xtrain.Mean_DeductibleAmtPaid[~Xtrain.Mean_DeductibleAmtPaid.isna()])), inplace=True)

In [126]:
np.sum(Xtrain.isna()).sort_values(ascending=False).head(6)

Perc_ClaimStartDt_December          0
Mean_Perc_of_AnnualDeductible       0
Mean_OPAnnualReimbursementAmt       0
Mean_OPAnnualDeductibleAmt          0
Mean_Bene_Age_at_ClaimStartDt       0
Mean_TotalAnnualReimbursementAmt    0
dtype: int64

Eliminating Infinite Values

In [124]:
np.sum(np.isinf(Xtrain)).sort_values(ascending=False).head()

Mean_Perc_of_AnnualReimbursement    12
Perc_ClaimStartDt_December           0
Mean_Perc_of_AnnualDeductible        0
Mean_OPAnnualReimbursementAmt        0
Mean_OPAnnualDeductibleAmt           0
dtype: int64

In [131]:
badindices = np.where(np.isinf(Xtrain))
Xtrain.iloc[-badindices[0], 24].median()
Xtrain.iloc[badindices[0], 24] = Xtrain.iloc[-badindices[0], 24].median()
np.sum(np.isinf(Xtrain)).sort_values(ascending=False).head()

Perc_ClaimStartDt_December       0
Mean_Perc_of_AnnualDeductible    0
Mean_OPAnnualReimbursementAmt    0
Mean_OPAnnualDeductibleAmt       0
Mean_Bene_Age_at_ClaimStartDt    0
dtype: int64

Final check that all features are in numerical format

In [134]:
np.sum(Xtrain.select_dtypes(exclude=np.number))

Series([], dtype: float64)

Separating Xtrain from Xtest and creating ytrain and ytest

In [143]:
Xtrain_all = Xtrain.copy()

In [135]:
provider_train = list(Train_Fraud_Samples.set_index('Provider').index)
provider_test = list(Test_Fraud_Samples.set_index('Provider').index)

In [146]:
Xtest = Xtrain.loc[provider_test]
Xtrain = Xtrain.loc[provider_train]

In [148]:
ytrain = Train_Fraud_Samples.set_index('Provider')
ytrain.PotentialFraud.replace('Yes','1', inplace=True)
ytrain.PotentialFraud.replace('No','0', inplace=True)
ytrain.PotentialFraud = pd.to_numeric(ytrain.PotentialFraud)

ytest = Test_Fraud_Samples.set_index('Provider')

In [153]:
print(Train_Fraud_Samples.shape[0], ytrain.shape[0], Xtrain.shape)
print(Test_Fraud_Samples.shape[0], ytest.shape[0], Xtest.shape)

5410 5410 (5410, 71)
1353 1353 (1353, 71)


In [155]:
with open('All_Data_Grouped.pickle', 'wb') as f:
    pickle.dump(All_Data, f)
    
with open('Xdata.pickle', 'wb') as f:
    pickle.dump(All_Visits_Bene_Important, f)
    
with open('Xtrain.pickle', 'wb') as f:
    pickle.dump(Xtrain, f)
with open('Xtest.pickle', 'wb') as f:
    pickle.dump(Xtest, f)
with open('ytrain.pickle', 'wb') as f:
    pickle.dump(ytrain, f)
with open('ytest.pickle', 'wb') as f:
    pickle.dump(ytest, f)

In [161]:
Xtrain

,Number_of_Claims,Num_Unique_Patients,Num_Unique_Docs,Num_Unique_Diag_Codes,Num_Unique_Proc_Codes,Mean_InscClaimAmtReimbursed,Mean_DeductibleAmtPaid,Mean_Reimbursement_per_Day,Mean_Length_of_Stay,Mean_Length_of_Claim,...,Perc_ClaimStartDt_March,Perc_ClaimStartDt_April,Perc_ClaimStartDt_May,Perc_ClaimStartDt_June,Perc_ClaimStartDt_July,Perc_ClaimStartDt_August,Perc_ClaimStartDt_September,Perc_ClaimStartDt_October,Perc_ClaimStartDt_November,Perc_ClaimStartDt_December
Provider,,,,,,,,,,,,,,,,,,,,,
PRV51001,25,24,19,72,3,4185.60,213.60,3700.00,6.00,2.44,...,0.12,0.08,0.20,0.12,0.08,0.04,0.04,0.00,0.00,0.08
PRV51003,132,117,51,365,39,4588.41,502.17,2137.42,6.16,4.67,...,0.08,0.08,0.06,0.11,0.08,0.10,0.07,0.08,0.08,0.05
PRV51004,149,138,48,267,0,350.13,2.08,0.00,0.00,2.43,...,0.12,0.07,0.05,0.07,0.09,0.07,0.08,0.07,0.05,0.11
PRV51005,1165,495,7,1294,0,241.12,3.18,0.00,0.00,2.09,...,0.10,0.08,0.07,0.09,0.08,0.08,0.09,0.10,0.06,0.07
PRV51007,72,58,11,165,1,468.19,45.33,1080.56,6.33,1.96,...,0.12,0.06,0.12,0.12,0.04,0.08,0.10,0.03,0.08,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PRV57759,28,24,1,49,0,380.00,4.64,0.00,0.00,3.14,...,0.07,0.07,0.14,0.11,0.11,0.04,0.07,0.04,0.07,0.04
PRV57760,22,9,3,47,0,216.82,0.00,0.00,0.00,1.32,...,0.05,0.05,0.05,0.14,0.00,0.18,0.14,0.00,0.09,0.05
PRV57761,82,67,7,182,0,225.24,4.51,0.00,0.00,2.39,...,0.05,0.06,0.10,0.12,0.06,0.06,0.05,0.17,0.13,0.06
